**Overview:**  
The NASA Goddard Earth Sciences Data and Information Services Center (GES DISC) has developed an Application Program Interface (API) for interacting with our Web Processing Services in a programmatic way. The API is intended for users who would like to apply our subsetting services to numerous data granules spanning a long time range or a variety of data products -- circumstances that make using the Web browser interface quite inefficient.

**Example:**  
This example code demonstrates how to use the API to submit an asynchronous request to the GES DISC Subsetting Service using Python3. The API is a communication protocol that allows users to find the granules they need and download the desired data subsets. Information is passed back and forth in JavaScript Object Notation (JSON) format.

**Prerequisites:**  
This example code is written in Python3 and requires these libraries: sys, json, urllib3, certifi, requests, time.

The first step is to import the required Python libraries. If any of the following import commands fail, check the local Python environment and install any missing packages. These lines will be necessary to run the rest of the cells:

In [1]:
import sys
import json
import urllib3
import certifi
import requests
from time import sleep

Initialize the urllib PoolManager and set the base URL for the API requests that will be sent to the GES DISC subsetting service.

In [2]:
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

# Set the URL for the GES DISC subset service endpoint
svcurl = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

Define a local general-purpose method that submits JSON-formatted WSP requests to the GES DISC server, checks for any errors, and then returns the service’s response. This method is created for convenience since this task will be repeated more than once. 

In [3]:
# This method POSTs formatted JSON WSP requests to the GES DISC endpoint URL and returns the response
def get_http_data(request):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', svcurl, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty request')
    return response

The data product used in this example is the Microwave Limb Sounder Level 2 Temperature Profile (ML2T_004). Three variables are selected: Temperature, TemperaturePrecision, and the Quality flag. The spatial domain is the global latitude band from 30oS to 30oN, vertical pressure levels range from 1000 to 100 hectoPascals (hPa), and the date range is 1-3 August 2015. The specifics of the subset are coded as local variables so they can be easily changed for different use cases. The desired spatial and temporal constraints, along with the dataset and variable specifications, are stored in a JSON-based Web Service Protocol (WSP) structure, which is named “subset_request”. 

In [4]:
# Define the parameters for the data subset
product = 'S5P_L2__NO2____HiR_2'
begTime = '2022-07-06T00:00:00Z'
endTime = '2022-07-06T23:59:59Z'
minlon = 21.4
maxlon = 41.01
minlat = 44.2
maxlat = 52.6
varNames = ['/PRODUCT/nitrogendioxide_tropospheric_column']



# The dimension slice will be for pressure levels between 1000 and 100 hPa
#dimName = '/HDFEOS/SWATHS/Temperature/nLevels'
#dimVals = [1,2,3,4,5,6,7,8,9,10,11,12,13] 
#dimSlice = []
#for i in range(len(dimVals)) :
  #  dimSlice.append({'dimensionId': dimName, 'dimensionValue': dimVals[i]})

The parameters in this particular subset_request structure are: <code>methodname</code>, <code>type</code>, <code>version</code>, and <code>args</code>. The <code>args</code> contain additional parameters that control the specifics for the subset.  For this example, the args parameters are: <code>role</code>, <code>start</code>, <code>end</code>, <code>box</code>, <code>crop</code>, and <code>data</code>. The <code>start</code> and <code>end</code> parameters provide the desired time range. The <code>box</code> parameter specifies the desired spatial domain which will constrain the granule search -- only data granules that cover the domain will be returned. The <code>crop</code> parameter is a True/False flag indicating whether to perform spatial subsetting on the granules returned by the spatial search. Granules will not be trimmed to the specified spatial domain unless <code>crop</code> is set to True. The <code>data</code> parameter is another list containing attribute:value pairs that include the <code>datasetID</code>, the <code>variable</code> name, and the <code>slice</code> parameter, which contains a list of dimensionName:index pairs. Each desired variable must be listed separately within the <code>data</code> parameter. To retrieve all the variables in the data file, omit the variable:name pair. The <code>slice</code> parameter is also optional; leave it out to retrieve all the variable dimensions.

In [5]:
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],  
        'crop'  : True,
        'data'  : [{'datasetId': product,
                    'variable' : varNames[0]
                   }]
                   
    }
}

For a point+radius subset, use <code>lon</code>, <code>lat</code>, and <code>radius</code> parameters instead of <code>box</code>. For example, these values might be suitable for selecting radial subsets around Greenland:

        'lon'    : -40.0,
        'lat'    : 72.0,
        'radius' : '12deg',


In the next step, the JSON-formatted subset_request is POSTed to the GES DISC server. 
The Job ID is extracted from the response -- to be used later as a reference for the request.

In [6]:
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)
print(json.dumps(response))
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

{"type": "jsonwsp/response", "version": "1.0", "servicename": "UUI subsetting service", "method": "subset", "result": {"PercentCompleted": 0, "Status": "Accepted", "jobId": "62e161f5a8a7141a4898fb4d", "sessionId": "62e161f5a8a7141a4898fb4c", "message": "Processing (S5P_L2__NO2____HiR_2)", "updated": "2022-07-27T16:04:05.427Z"}}
Job ID: 62e161f5a8a7141a4898fb4d
Job status: Accepted


At this point in the code, the job is running on the GES DISC server. The next step is to construct another JSON WSP request to periodically retrieve the job status, using the extracted Job ID. When the job is finished, check on the final status to ensure the job succeeded. 

In [7]:
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
    
print(response['result'])
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Job status: Succeeded (100% complete)
{'PercentCompleted': 100, 'Status': 'Succeeded', 'jobId': '62e161f5a8a7141a4898fb4d', 'sessionId': '62e161f5a8a7141a4898fb4c', 'message': 'Complete (S5P_L2__NO2____HiR_2)', 'updated': '2022-07-27T16:04:09.229Z'}
Job Finished:  Complete (S5P_L2__NO2____HiR_2)


Knowing that the job has finished successfully, it is time to retrieve the results. The results of a subset request job are URLs: there are HTTP_Services URLs (one for every data granule in the time range of interest) plus links to any relevant documentation. Each HTTP_Services URL contains the specifics of the subset request encoded as facets. Data subsets and documentation files are downloaded using the requests Python library.

There are two ways to retrieve the list of URLs when the subset job is finished:

**Plan A:** 
Use the API method named GetResult. This method will return the URLs along with three additional attributes: a label, plus the beginning and ending time stamps for that particular data granule. The label serves as the filename for the downloaded subsets. 

**Plan B:**
Retrieve a plain-text list of URLs in a single shot using the saved JobID. This is a shortcut to retrieve just the list of URLs without any of the other metadata. 

Below is the code for **Plan A**.
The steps are to construct a third type of JSON WSP request that retrieves the results of this Job. When that request is submitted, the results are returned in multiple batches of 20 items, starting with item 0. The startIndex value in the results_request structure must be updated after each successive batch is retrieved.

In [8]:
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
        
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 7 out of 7 expected items


Below is the code for **Plan B**. 
Construct a request using the saved JobID and retrieve the results with the requests library. If the requests.get() method does not return an error, the URLs are stored locally and printed out for informational purposes. 

In [9]:
# Retrieve a plain-text list of results in a single shot using the saved JobID
result = requests.get('https://disc.gsfc.nasa.gov/api/jobs/results/'+myJobId)
try:
    result.raise_for_status()
    urls = result.text.split('\n')
    for i in urls : print('\n%s' % i)
except :
    print('Request returned error code %d' % result.status_code)


https://sentinel.esa.int/documents/247904/2476257/Sentinel-5P-TROPOMI-ATBD-NO2-data-products

https://sentinel.esa.int/documents/247904/3541451/Sentinel-5P-Nitrogen-Dioxide-Level-2-Product-Readme-File

https://tropomi.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FS5P_TROPOMI_Level2%2FS5P_L2__NO2____HiR.2%2F2022%2F186%2FS5P_OFFL_L2__NO2____20220705T233045_20220706T011215_24496_02_020301_20220707T151954.nc&FORMAT=bmM0Lw&BBOX=44.2%2C21.4%2C52.6%2C41.01&TIME=2022-07-06T00%3A00%3A00%2F2022-07-06T23%3A59%3A59&LABEL=S5P_OFFL_L2__NO2____20220705T233045_20220706T011215_24496_02_020301_20220707T151954.SUB.nc4&FLAGS=GRIDTYPE__SWATH&SHORTNAME=S5P_L2__NO2____HiR&SERVICE=SUBSET_LEVEL2&VERSION=1.02&DATASET_VERSION=2&VARIABLES=..2FPRODUCT..2Fnitrogendioxide_tropospheric_column

https://tropomi.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FS5P_TROPOMI_Level2%2FS5P_L2__NO2____HiR.2%2F2022%2F187%2FS5P_OFFL_L2__NO2____20220706T075814_20220706T093944

It is important to remember that the results returned at this point are not data files, but lists of URLs. Most of the URLs will contain HTTP_services requests to actually do the subsetting and return the data, but some of them may be links to documentation files pertaining to the dataset in question. It is worthwhile to separate the document URLs from the HTTP_services URLs in case the documentation has already been retrieved. The way we do this is to check for start and end attributes, which are always associated with HTTP_services URLs. 

The remainder of the example code assumes the use of **Plan A** because it makes use of this extra metadata. 

In [10]:
# Sort the results into documents and URLs
docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)

# Print out the documentation links, but do not download them
print('\nDocumentation:')
for item in docs : print(item['label']+': '+item['link'])


Documentation:
Algorithm Theoretical Basis Document: https://sentinel.esa.int/documents/247904/2476257/Sentinel-5P-TROPOMI-ATBD-NO2-data-products
README Document: https://sentinel.esa.int/documents/247904/3541451/Sentinel-5P-Nitrogen-Dioxide-Level-2-Product-Readme-File


The final step is to use the requests.get() method to invoke each HTTP_Services URL and download the data files. The contents of the label attribute are used here as the output file name, but the name can be any string. It is important to download each file one at a time, in series rather than in parallel, in order to avoid overloading the GES DISC servers. 


In [11]:
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn)
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')


HTTP_services output:
S5P_OFFL_L2__NO2____20220705T233045_20220706T011215_24496_02_020301_20220707T151954.SUB.nc4
S5P_OFFL_L2__NO2____20220706T075814_20220706T093944_24501_02_020301_20220707T235214.SUB.nc4
S5P_OFFL_L2__NO2____20220706T093944_20220706T112114_24502_02_020301_20220708T015012.SUB.nc4
S5P_OFFL_L2__NO2____20220706T112114_20220706T130243_24503_02_020301_20220708T033431.SUB.nc4
S5P_OFFL_L2__NO2____20220706T231142_20220707T005312_24510_02_020301_20220708T145712.SUB.nc4


If the code above does not succeed in your particular environment, please check the [Earthdata wiki page](https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+Python) for alternative Python examples. [The GES DISC guide to data access](https://disc.gsfc.nasa.gov/data-access) has some additional options for downloading data URLs. 

**Additional Info:**  
[Complete reference documentation for the GES DISC Subsetting Service API](https://disc.gsfc.nasa.gov/service/subset)  

<font size="1">THE SUBJECT FILE IS PROVIDED "AS IS" WITHOUT ANY WARRANTY OF ANY KIND, EITHER EXPRESSED, IMPLIED, OR STATUTORY, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTY THAT THE SUBJECT FILE WILL CONFORM TO SPECIFICATIONS, ANY IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR FREEDOM FROM INFRINGEMENT, ANY WARRANTY THAT THE SUBJECT FILE WILL BE ERROR FREE, OR ANY WARRANTY THAT DOCUMENTATION, IF PROVIDED, WILL CONFORM TO THE SUBJECT FILE. THIS AGREEMENT DOES NOT, IN ANY MANNER, CONSTITUTE AN ENDORSEMENT BY GOVERNMENT AGENCY OR ANY PRIOR RECIPIENT OF ANY RESULTS, RESULTING DESIGNS, HARDWARE, SOFTWARE PRODUCTS OR ANY OTHER APPLICATIONS RESULTING FROM USE OF THE SUBJECT FILE. FURTHER, GOVERNMENT AGENCY DISCLAIMS ALL WARRANTIES AND LIABILITIES REGARDING THIRD-PARTY SOFTWARE, IF PRESENT IN THE SUBJECT FILE, AND DISTRIBUTES IT "AS IS."</font>